In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from notion_client import Client
import time
from tools import *
from make_data import load_dict_from_json

# Load environment variables
load_dotenv()
notion_auth = os.getenv('NOTION_API_KEY')
notion_db = os.getenv('NOTION_DATABASE_ID')

In [33]:
database = load_dict_from_json('data/M0A_train_data.json')

In [34]:
themes = {
    "T1": "Conscience",
    "T2": "Desire",
    "T3": "Freedom",
    "T4": "Goodness",
    "T5": "Identity",
    "T6": "Justice",
    "T7": "Language",
    "T8": "Meaning",
    "T9": "Science",
    "T10": "Technology",
    "T11": "Truth",
    "T12": "Time",
    "T13": "Existence",
    "T14": "Music",
    "T15": "Imagination",
    "T16": "The Unconscious",
    "T17": "Education",
    "T18": "Body & Mind",
    "T19": "Beauty",
    "T20": "Art",
    "T21": "Love",
    "T22": "Reality",
    "T23": "Politics",
    "T24": "Work",
    "T25": "Living Together",
    "T26": "Philosophy",
    "T27": "Matter",
    "T28": "Death",
    "T29": "Religion",
    "T30": "History",
    "T31": "Thought",
    "T32": "Madness",
    "T33": "Joy & Happiness"
}

In [35]:
prompt = """
> You are given the text of a page from a digital book and a fixed list of 33 themes.
Your task is to determine which of these themes are relevant to the content of the page.
A page can be allocated to more than one theme. Return the allocation in the format:
[T<theme_number>, T<theme_number>, ...]
>
>
> Below are the 33 themes along with their guiding questions and descriptions:
>
> T1. Conscience
> Does consciousness make man an exception?
> Is an individual's conscience merely a reflection of the society to which he belongs?
>
> T2. Desire
> To be passionate is to harm oneself?
> Can desire be satisfied with reality?
> Should we follow our desire or liberate ourselves from it?
> Can one desire without suffering?
> Do we only want the things we think are good?
> Can we say of a desire that it is abnormal?
> Is it absurd to desire the impossible?
> Does desire compel us to experience it?
> Does passion prevent us from doing our duty?
> Does desire take us astray from the truth?
> Do we always know what we want?
>
> T3. Freedom
> Is becoming aware liberating?
> Can we choose to be free?
> Is to oppose authority always a sign of freedom?
> Does the idea of total freedom make sense?
> To be free, is it to meet no obstacle?
> Why do we want to be free?
> How much of character is innate and how much is acquired?
> Does luck exist?
>
> T4. Goodness
> Does being educated makes one a better person?
> Is the determination of goodness only a matter of opinion?
> Is it enough to see the best in order to follow it?
> Can one want what is good without doing it?
> Does speaking of inhuman actions make any sense?
> How does one define goodness?
> Can moral sentiment be taught?
>
> T5. Identity
> To change is to become someone else?
> Is to be self-aware becoming alien to oneself?
> Is it better to know oneself?
> Am I or do I become myself?
> Am I the subject of my thoughts?
> Can we be ourselves in front of others?
> Does work enable one to become aware of oneself?
> Do others teach me something about myself?
> Are we solely the sum of the choices we make?
>
> T6. Justice
> Are the requirements for justice and freedom separable?
> Is the law merely a justice by default?
> Is what is just and unjust merely a matter of conventions?
> Does experimenting injustice teach me what is right?
> Is justice solely a prerogative of the state?
> Is it better to commit or to be a victim of injustice?
> Is democracy a guaranty of just laws?
>
> T7. Language
> Can we say that language hampers thought?
> Does language betray thought?
> By using language do we renounce recurring to violence?
> Is it necessary to speak to make oneself understood?
> Is communication the only use for language?
> Are there things that language cannot tell?
> Is language only a tool?
>
> T8. Meaning
> What has meaning?
>
> T9. Science
> Is scientific knowledge of life possible?
> Are there questions that no science can answer?
> Can experience prove something?
> Can we believe without knowing?
> Comment on this thought of Nietzsche: "It is not doubt, but certainty that drives us mad."
> Are the principles of reason a product of experience?
>
> T10. Technology
> Is there more to hope than to fear from technology?
> To respect nature is to renounce transforming it?
> Does technological development transform mankind?
> Would we be freer without machines?
> Is mankind at home in nature?
>
> T11. Truth
> Does the truth depend on us?
> How can we recognize the truth?
> Does science derive solely from our desire for knowing the truth?
> Is plurality of opinions an obstacle to the truth?
> Are there other means than demonstration to establish the truth?
> What need do we have to seek the truth?
> Can we be sure of being right?
> Is there no truth at all in a lie?
> Can what is true in theory be false in practice?
> Should we be wary of multiplicity of interpretations?
> Doubt: A strength or a weakness?
> Are all beliefs contrary to reason?
> Do we interpret as we come short of knowing?
> Are all beliefs worth the same?
> Does politics escape from the requirement of truth?
> Can one be right against the facts?
> Can one not admit the truth?
>
> T12. Time
> Is free time the time of my freedom?
> Does time destroy everything?
> Can we only be happy in the past tense?
> Does it make sense to try escaping from time?
> Does the future only exist in our mind?
> Is time the limit of mankind?
> Do we know the present better than the past?
> Is taking one's time a waste?
> How can there be something new?
> Is time a linear process?
> Is partying always futile?
>
> T13. Existence
> To exist, is it to act?
> Can we understand what the origin is in thought?
> Is existence about enjoying the moment?
> What is a fruitful day?
> Do we risk missing out on life?
>
> T14. Music
>
> T15. Imagination
> Does imagination enrich knowledge?
> What do we mean by lack of imagination?
>
> T16. The Unconscious
> Does the idea of the unconscious exclude that of freedom?
> To admit the existence of the unconscious is to give up on any effort of lucidity with regard to oneself?
> Can we say more than we think?
>
> T17. Education
> Is culture liberating?
> Does culture make us more human?
> Are all cultures worth the same?
> What is the proportion of the innate vs. the acquired in one's character?
> Is the more cultured you are, the happier you are?
> Can the moral sentiment be learned?
> Is culture a simple addition to nature?
>
> T18. Body & Mind
> What am I in relation to my body?
> What difference can be made between the mind and the body?
>
> T19. Beauty
> Does beauty transform our consciousness of reality?
> What is the relationship between beauty and goodness?
> Is there a natural beauty?
> Is there a privilege of beauty?
> Is beauty a promise of happiness?
> How is aesthetic feeling different from religious feeling?
> In general when a thing becomes useful does it lose its beauty?
>
> T20. Art
> Can one love a work of art without understanding it?
> Can art manifest the truth?
> Are works of art realities like the others?
> Can art transform our awareness of reality?
> Does sensitivity to works of art require education?
> Can art show what language can not say?
> How is artistic beauty superior to natural beauty?
> Why are artworks invented by the human mind more valuable than imitations?
> Can a work of art teach us something?
> Can a work of art escape the criteria of beauty and ugliness?
> Is art less necessary than science?
> Does one choose to be an artist?
> Is art a matter of personal taste?
> Does a work of art always have a meaning?
>
> T21. Love
> The heart has its reasons that reason ignores?
> What do we love in love?
> Is it reasonable to be in love?
> Can one love one's neighbor as oneself?
> Is it realistic to claim to be able to love all men?
> In all love do we love only ourselves?
> To love someone does one have to know him?
> How can one differentiate love from friendship?
>
> T22. Reality
> Can perception be learned?
> What can we know about reality?
> Are appearances deceptive?
> Can one rely on intuition?
> Can we perceive without judging?
> Do we systematically run away from reality?
> Can we distinguish dream from reality?
>
> T23. Politics
> Is man a political animal?
> Who is allowed to tell me "You must"?
>
> T24. Work
> Is necessity at the origin of labor?
> Does the division of labor separate people?
> Should we make a virtue out of work?
> Is working a loss of one's time?
> Is work only the implementation of technical skills?
>
> T25. Living Together
> Do men live in society out of necessity?
> Is respecting all living beings a moral obligation?
> Is solitude worthless?
> Is a selfless action possible?
> Is all violence without reason?
> Does passion prevent us from doing our duty?
> What do we have to gain by doing our duty?
> Why is an action moral?
> What can we know about others?
> Do I need to identify myself with someone in order to understand him?
> Can we conceive a society without conflict?
> Is one responsible only for one's own actions?
> Do we only have obligations towards others?
> Can we experiment on living beings?
> What do one gain from exchanging?
> How can there be a counter-power?
> Should happiness of citizens be the goal of politics?
>
> T26. Philosophy
> Is happiness the goal of philosophy?
> Should the philosopher govern?
> Can philosophy talk about religion?
>
> T27. Matter
> Does the mind have access to matter?
> What is matter?
>
> T28. Death
> What does death teach us?
> Must man resign himself to die?
> Can we understand what death is in thought?
>
> T29. Religion
> Does mankind necessarily need religion?
> Are all beliefs contrary to reason?
> Is religion just a product of culture?
>
> T30. History
> Is history a science?
> Do we need the past to build our future?
> Should one use knowledge of history to guide political action?
> Who makes history?
>
> T31. Thought
> Does our freedom of thought have limits?
> What is an idea?
> Can we understand in thought what the origin is?
>
> T32. Madness
> Comment on this thought of Nietzsche: "It is not doubt, but certainty that drives us mad."
>
> T33. Joy & Happiness
> Should we prefer happiness to the truth?
> Is happiness a private matter?
> Is unconsciousness the condition for happiness?
> Is it illusory to try to be happy?
> Is happiness the goal of philosophy?
> Can one be happy in an unfair world?
> Can one only be happy in the past?
> Can one desire without suffering?
> Is happiness the goal of existence?
> Is the more cultured you are, the happier you are?
> Is rest a condition of happiness?
>
> DO NOT provide explanations, only provide the allocation code line.
> DO NOT provide the title of each theme such as [T7. Language, T5. Identity], only provide the theme code such as [T7, T5].
>
> If uncertain, pick the most likely themes based on the textual content.
>
"""

In [36]:
delay_between_batches = 20  # Time (in seconds) to wait between batches
test_sample = select_random_keys(database, 10)
system_instructions = prompt

def process_in_batches(test_sample, delay_between_batches, system_instructions):

    results = {}

    for text in test_sample:
        allocation = get_theme_allocation(text, system_instructions).replace(']', '').replace('[', '').replace(' ', '').split(',')
        results[text] = [themes[allocation] for allocation in allocation]

        time.sleep(delay_between_batches)
    return results

results_dict = process_in_batches(test_sample, delay_between_batches, system_instructions)

In [37]:
save_answers_to_sqlite('data/models_answers.db', 'test', results_dict, database)

In [ ]:
y_pred = create_binary_matrix(themes, [item for key, item in results_dict.items()])
y_true = create_binary_matrix(themes, [database[key] for key in test_sample])

jaccard_index, hamming_loss = evaluate_metrics(y_true, y_pred)
personalized_loss = personalized_metric(y_true, y_pred)

In [4]:
prompt_metrics_save('prompt_name', prompt, jaccard_index, hamming_loss, personalized_loss)